In [1]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from collections import deque
import matplotlib.pyplot as plt

In [2]:
class A2C(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(A2C, self).__init__()
        self.common = Dense(128, activation='relu')
        self.actor  = Dense(action_size, activation='softmax',
                                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        self.critic = Dense(1,
                                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        
    def call(self, x):
        x      = self.common(x)
        policy = self.actor(x)
        value  = self.critic(x)
        return policy, value

In [3]:
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper params for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        
        self.model = A2C(self.state_size,self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
    def get_action(self, state):
        policy, _ = self.model(state)
        policy = np.array(policy[0])
        return np.random.choice(self.action_size, 1, p=policy)[0]
        
    def train_model(self, state, action, reward, next_state, done):
        model_params = self.model.trainable_variables
        # print(model_params)
        with tf.GradientTape() as tape:
            policy, value      = self.model(state)
            _,      next_value = self.model(next_state)
            target = reward + (1 - done) * self.discount_factor * next_value[0]
            
            # For policy network
            one_hot_action = tf.one_hot([action], self.action_size)
            action_prob = tf.reduce_sum(one_hot_action * policy, axis=1)
            cross_entropy = - tf.math.log(action_prob + 1e-5)
            advantage = tf.stop_gradient(target - value[0])
            actor_loss = tf.reduce_mean(cross_entropy * advantage)
            
            # For value network
            critic_loss = 0.5 * tf.square(tf.stop_gradient(target) - value[0])
            critic_loss = tf.reduce_mean(critic_loss)
            
            # integrate losses
            loss = 0.2 * actor_loss + critic_loss
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        return np.array(loss)

In [4]:
%matplotlib tk
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = A2CAgent(state_size, action_size)

    scores, episodes, losses = [], [], []
    score_avg = 0
    
    end = False
    
    fig = plt.figure(1)
    fig.clf()
    
    num_episode = 2000
    for e in range(num_episode):
        done = False
        score = 0
        loss_list = []
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        
        while not done:
            # env.render()

            action = agent.get_action(state)

            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            score += reward
            reward = 0.1 if not done or score == 500 else -1

            loss = agent.train_model(state, action, reward, next_state, done)
            loss_list.append(loss)

            state = next_state
            if done:
                
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | loss: {:.4f}'.format(e, score_avg, np.mean(loss_list)))

                scores.append(score_avg)
                episodes.append(e)
                losses.append(np.mean(loss_list))
                plt.subplot(211)
                plt.plot(episodes, scores, 'b')
                plt.xlabel('episode')
                plt.ylabel('average score')
                plt.title('cartpole A2C')
                
                plt.subplot(212)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode')
                plt.ylabel('loss')
                
                plt.savefig('./save_model/cartpole_a2c.png')

                if score_avg > 400:
                    agent.model.save_weights('./save_model/cartpole_a2c', save_format='tf')
                    end = True
                    break
        if end == True:
            np.save('./save_model/cartpole_a2c_epi',episodes)
            np.save('./save_model/cartpole_a2c_score',scores)
            np.save('./save_model/cartpole_a2c_loss',losses)
            env.close()
            print("End")
            break

epi:   0 | score avg 14.00 | loss: 0.0565
epi:   1 | score avg 16.80 | loss: 0.0305
epi:   2 | score avg 18.52 | loss: 0.0448
epi:   3 | score avg 18.07 | loss: 0.0664
epi:   4 | score avg 18.56 | loss: 0.1428
epi:   5 | score avg 19.51 | loss: 0.2189
epi:   6 | score avg 18.75 | loss: 0.0605
epi:   7 | score avg 19.78 | loss: 0.0402
epi:   8 | score avg 19.20 | loss: 0.0629
epi:   9 | score avg 18.68 | loss: 0.2825
epi:  10 | score avg 19.31 | loss: 0.1337
epi:  11 | score avg 18.28 | loss: 0.3149
epi:  12 | score avg 18.95 | loss: 0.1064
epi:  13 | score avg 18.56 | loss: 0.1590
epi:  14 | score avg 18.80 | loss: 0.1240
epi:  15 | score avg 18.02 | loss: 0.1568
epi:  16 | score avg 17.62 | loss: 0.1102
epi:  17 | score avg 17.06 | loss: 0.1385
epi:  18 | score avg 16.45 | loss: 0.1444
epi:  19 | score avg 17.81 | loss: 0.0603
epi:  20 | score avg 17.13 | loss: 0.1320
epi:  21 | score avg 19.91 | loss: 0.0478
epi:  22 | score avg 19.02 | loss: 0.1185
epi:  23 | score avg 19.12 | loss: